# Prerequirements

In [81]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import cv2
import glob
import imgaug.augmenters as iaa
import imgaug as ia
import datetime

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, BatchNormalization, Layer, ReLU, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K
from tensorboard.plugins.hparams import api as hp

import warnings
warnings.filterwarnings("ignore")

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-01-16 22:35:59.798719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 22:35:59.904697: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 22:35:59.905830: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
def imshow(img: np.array):
    if img.shape[0] * 2 > img.shape[1]:
        fig = plt.figure(figsize=(7, 7))
    else:
        fig = plt.figure(figsize=(20, 20))
    plt.axis('off')
    plt.imshow(img)

In [51]:
! mkdir ./tensorboard/
! mkdir ./tensorboard/autoencoder/
! mkdir ./tensorboard/autoencoder/fit/
! mkdir ./models/
! mkdir ./models/autoencoder/
! mkdir ./models/autoencoder/hparam_tuning

mkdir: cannot create directory ‘./tensorboard/’: File exists
mkdir: cannot create directory ‘./tensorboard/autoencoder/’: File exists
mkdir: cannot create directory ‘./tensorboard/autoencoder/fit/’: File exists
mkdir: cannot create directory ‘./models/’: File exists
mkdir: cannot create directory ‘./models/autoencoder/’: File exists


# Preparing dataset

In [ ]:
# ds = []
# for idx, file in enumerate(glob.glob("/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/*")):
#     ds.append(np.array(PIL.Image.open(file).resize((112, 96))))
#     if idx >= 10001: break
# ds = np.array(ds)
# ds.shape

In [ ]:
# idxs = np.random.choice(len(ds), 5)
# imshow(np.concatenate(ds[idxs], 1))

In [ ]:
# def create_line_mask(img):
#     mask = np.full(img.shape, 255, np.uint8)
#     for _ in range(np.random.randint(6, 10)):
#         x1, x2 = np.random.randint(1, img.shape[1]), np.random.randint(1, img.shape[1])
#         y1, y2 = np.random.randint(1, img.shape[0]), np.random.randint(1, img.shape[0])
#         thickness = np.random.randint(4, 6)
#         cv2.line(mask, (x1, y1), (x2, y2), (1, 1, 1), thickness)

#     masked_image = cv2.bitwise_and(img, mask)

#     return masked_image

In [ ]:
# idxs = np.random.choice(len(ds), 5)
# masked = np.array(list(map(create_line_mask, ds[idxs])))
# imshow(np.concatenate(masked, 1))

In [ ]:
# for idx, sample in enumerate(ds):
#     PIL.Image.fromarray(sample).save(f'./data/samples/{idx}.png')
#     PIL.Image.fromarray(create_line_mask(sample)).save(f'./data/samples_line_masked/{idx}.png')
#     PIL.Image.fromarray(create_line_mask(sample)).save(f'./data/samples_square_masked/{idx}.png')

# Custom generator

In [5]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, Y, batch_size=128, dim=(112, 96), n_channels=3): 
        self.X = X
        self.Y = Y
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels

        self.on_epoch_end()
        assert(len(self.X) == len(self.Y) or len(self.X) > 0)

        
    def __len__(self):
        return int(np.floor(len(self.X) / self.batch_size))


    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index+1) * self.batch_size]
        return self.__data_generation(indexes)

    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.X))
    

    def __data_generation(self, idxs):
        X_batch = np.empty((self.batch_size, self.dim[0], self.dim[1], self.n_channels))
        Y_batch = np.empty((self.batch_size, self.dim[0], self.dim[1], self.n_channels))

        for i, idx in enumerate(idxs):
            image = np.array(PIL.Image.open(self.X[idx]))
            label = np.array(PIL.Image.open(self.Y[idx]))
#             if np.random.randint(0, 100) < 20:
#                 X_batch[i,] = self.augment(image / 255)
#                 Y_batch[i,] = self.augment(label / 255)
#             else:
#                 X_batch[i,] = image / 255
#                 Y_batch[i,] = label / 255
            X_batch[i,] = image / 255
            Y_batch[i,] = label / 255

        return X_batch, Y_batch

    
    def augment(self, img):
        seq = iaa.Sequential([
            iaa.Sometimes(0.1, iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)))
        ])
        return seq(images=img)

In [6]:
def train_test_split(X, Y, train_size=0.8):   
    train_split = int(train_size * len(X))
    
    X_train = X[:train_split]
    Y_train = Y[:train_split]
    
    X_test = X[train_split:]
    Y_test = Y[train_split:]
    
    return X_train, X_test, Y_train, Y_test

In [9]:
X = sorted(glob.glob("/kaggle/input/cv-project3/data/samples_masked/*.png"))[:500]
Y = sorted(glob.glob("/kaggle/input/cv-project3/data/samples/*.png"))[:500]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [10]:
train_gen = DataGenerator(X_train, Y_train)
test_gen = DataGenerator(X_test, Y_test)

# Metrics

In [11]:
def dice_coef(y_true, y_pred):
    y_true_f = keras.backend.flatten(y_true)
    y_pred_f = keras.backend.flatten(y_pred)
    intersection = keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (keras.backend.sum(y_true_f + y_pred_f))

In [12]:
def jaccard_distance(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [14]:
cosine_similarity = tf.keras.metrics.CosineSimilarity(axis=1)

# Autoencoder

In [82]:
class Autoencoder(keras.Model):
    def __init__(self):
        super(Autoencoder, self).__init__()


    def __ConvBlock(self, out, kernel_size, prev_layer):
        cnn = Conv2D(out, kernel_size, padding="same")(prev_layer)
        cnn = BatchNormalization()(cnn)
        cnn = ReLU()(cnn)
        return cnn

    def __EncodeBlock(self, out, kernel_size, prev_layer, dr_rate=0.1):
        conv = self.__ConvBlock(out, kernel_size, prev_layer)
        conv = self.__ConvBlock(out, kernel_size, conv)
        conv = self.__ConvBlock(out, kernel_size, conv)
        conv = MaxPooling2D((2, 2))(conv)
        conv = Dropout(dr_rate)(conv)
        return conv


    def __DecodeBlock(self, out, kernel_size, prev_layer):
        up = Conv2DTranspose(out, kernel_size, strides=(2, 2), padding="same")(prev_layer)
        up = BatchNormalization()(up)
        up = ReLU()(up)
        return up


    def model(self, input_shape=(112, 96, 3), dr_rate=0.1):
        inputs = keras.layers.Input(input_shape)

        conv1 = self.__EncodeBlock(32, kernel_size, inputs, dr_rate) 
        conv2 = self.__EncodeBlock(64, kernel_size, conv1, dr_rate)
        conv3 = self.__EncodeBlock(128, kernel_size, conv2, dr_rate) 
        conv4 = self.__EncodeBlock(256, kernel_size, conv3, dr_rate) 

        deconv1 = self.__DecodeBlock(256, kernel_size, conv4)
        deconv2 = self.__DecodeBlock(128, kernel_size, deconv1)
        deconv3 = self.__DecodeBlock(64, kernel_size, deconv2)
        deconv4 = self.__DecodeBlock(32, kernel_size, deconv3)

        outputs = keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(deconv4)

        return keras.models.Model(inputs=[inputs], outputs=[outputs])

## Parameters tuning on small part of dataset

In [83]:
HP_DROPOUT = hp.HParam("dropout", hp.Discrete([0.1, 0.2, 0.3, 0.5]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'adagrad']))

METRIC = 'mean_absolute_error'

with tf.summary.create_file_writer('./tensorboard/autoencoder/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC, display_name='mean_absolute_error')]
    )

In [84]:
def train_test_model(hparams):
    model = Autoencoder().model(input_shape=(112, 96, 3), dr_rate=hparams[HP_DROPOUT])
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='mean_absolute_error',
    )

    history = model.fit(
        train_gen, 
        validation_data = test_gen, 
        epochs=10, 
        steps_per_epoch = len(train_gen), 
        validation_steps = len(test_gen),
        use_multiprocessing = True,
    )
    
    loss = model.evaluate(test_gen[0][0], test_gen[0][1])
    return loss

In [85]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        loss = train_test_model(hparams)
        tf.summary.scalar(METRIC, loss, step=1)

In [86]:
session_num = 0

for dr_rate in HP_DROPOUT.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        hparams = {
            HP_DROPOUT: dr_rate,
            HP_OPTIMIZER: optimizer,
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1

--- Starting trial: run-0
{'dropout': 0.1, 'optimizer': 'adagrad'}
Epoch 1/10
3/3 [==============================] - 5s 322ms/step - loss: 0.2886
Epoch 2/10
3/3 [==============================] - 3s 758ms/step - loss: 0.2884
Epoch 3/10
3/3 [==============================] - 4s 1s/step - loss: 0.2883
Epoch 4/10
3/3 [==============================] - 4s 1s/step - loss: 0.2880
Epoch 5/10
3/3 [==============================] - 4s 967ms/step - loss: 0.2877
Epoch 6/10
3/3 [==============================] - 3s 848ms/step - loss: 0.2875
Epoch 7/10
3/3 [==============================] - 3s 703ms/step - loss: 0.2874
Epoch 8/10
3/3 [==============================] - 5s 1s/step - loss: 0.2871
Epoch 9/10
3/3 [==============================] - 4s 1s/step - loss: 0.2868
Epoch 10/10
4/4 [==============================] - 1s 27ms/step - loss: 0.3246
--- Starting trial: run-1
{'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/10
3/3 [==============================] - 4s 754ms/step - loss: 0.2932
Epoch 2/10
3

Process Keras_worker_ForkPoolWorker-118:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.7/site-packages/keras/utils/data_utils.py", line 563, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/tmp/ipykernel_23/2284451755.py", line 19, in __getitem__
    return self.__data_generation(indexes)
  File "/tmp/ipykernel_23/2284451755.py", line 32, in __data_generation
    label = np.array(PIL.Image.open(self.Y[idx]))
  File "/opt/conda/lib/python3.7/site-packages/PIL/Image.py", line 719, in __array__
    new["data"] = self.tobytes()
  File "/opt/conda/lib/python3.7/site-packages/PIL/Image.py", line 762, in tobytes
    self.loa

KeyboardInterrupt: 

In [14]:
# model = Autoencoder().model(input_shape=(112, 96, 3))
# model.compile(optimizer='adam', loss='mean_absolute_error', metrics=[dice_coef, jaccard_distance, cosine_similarity])
# # keras.utils.plot_model(model, show_shapes=True, to_file='./autoencoder.png')

# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001)

In [300]:
# log_dir = "./tensorboard/autoencoder/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

2023-01-16 21:55:52.410415: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2023-01-16 21:55:52.410484: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2023-01-16 21:55:52.693307: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2023-01-16 21:55:52.693501: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


In [ ]:
# history = model.fit(
#     train_gen, 
#     validation_data = test_gen, 
#     epochs=1000, 
#     steps_per_epoch = len(train_gen), 
#     validation_steps = len(test_gen),
#     use_multiprocessing = True,
# #     callbacks=[tensorboard_callback]
# )

2023-01-16 22:26:23.876532: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000


2023-01-16 22:26:28.294558: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


31/31 [==============================] - 50s 1s/step - loss: 0.1445 - dice_coef: 0.5773 - jaccard_distance: 1.1066 - cosine_similarity: 0.9165 - val_loss: 0.2276 - val_dice_coef: 0.5488 - val_jaccard_distance: 1.3417 - val_cosine_similarity: 0.8924
Epoch 2/1000
31/31 [==============================] - 27s 865ms/step - loss: 0.0921 - dice_coef: 0.6141 - jaccard_distance: 0.9944 - cosine_similarity: 0.9536 - val_loss: 0.2099 - val_dice_coef: 0.5183 - val_jaccard_distance: 1.3320 - val_cosine_similarity: 0.8960
Epoch 3/1000
31/31 [==============================] - 25s 799ms/step - loss: 0.0832 - dice_coef: 0.6189 - jaccard_distance: 0.9826 - cosine_similarity: 0.9596 - val_loss: 0.1794 - val_dice_coef: 0.5143 - val_jaccard_distance: 1.2594 - val_cosine_similarity: 0.9090
Epoch 4/1000
31/31 [==============================] - 27s 864ms/step - loss: 0.0736 - dice_coef: 0.6239 - jaccard_distance: 0.9710 - cosine_similarity: 0.9641 - val_loss: 0.1777 - val_dice_coef: 0.4965 - val_jaccard_dista

Process Keras_worker_ForkPoolWorker-15:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 127, in worker
    put((job, i, result))
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 358, in put
    obj = _ForkingPickler.dumps(obj)
  File "/opt/conda/lib/python3.7/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
KeyboardInterrupt
Process Keras_worker_ForkPoolWorker-16:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessin

In [ ]:
# model.save("./models/autoencoder/")

In [ ]:
# for i in range(10):
#     gr_tr = test_gen[1][1][i]
#     pred = model.predict(np.array([test_gen[1][0][i]])).reshape(112, 96, 3)
#     res = np.concatenate([gr_tr, test_gen[1][0][i], pred], axis=1)
#     imshow(res)
# #     Image.fromarray(A)
# #     im.save("your_file.jpeg")

In [ ]:
# class inpaintingModel:
#   '''
#   Build UNET like model for image inpaining task.
#   '''
#   def prepare_model(self, input_size=(208, 176, 3)):
#     inputs = keras.layers.Input(input_size)

#     conv1, pool1 = self.__ConvBlock(32, (3,3), (2,2), 'relu', 'same', inputs) 
#     conv2, pool2 = self.__ConvBlock(64, (3,3), (2,2), 'relu', 'same', pool1)
#     conv3, pool3 = self.__ConvBlock(128, (3,3), (2,2), 'relu', 'same', pool2) 
#     conv4, pool4 = self.__ConvBlock(256, (3,3), (2,2), 'relu', 'same', pool3) 
    
#     conv5, up6 = self.__UpConvBlock(512, 256, (3,3), (2,2), (2,2), 'relu', 'same', pool4, conv4)
#     conv6, up7 = self.__UpConvBlock(256, 128, (3,3), (2,2), (2,2), 'relu', 'same', up6, conv3)
#     conv7, up8 = self.__UpConvBlock(128, 64, (3,3), (2,2), (2,2), 'relu', 'same', up7, conv2)
#     conv8, up9 = self.__UpConvBlock(64, 32, (3,3), (2,2), (2,2), 'relu', 'same', up8, conv1)
    
#     conv9 = self.__ConvBlock(32, (3,3), (2,2), 'relu', 'same', up9, False)
    
#     outputs = keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(conv9)

#     return keras.models.Model(inputs=[inputs], outputs=[outputs])  

#   def __ConvBlock(self, filters, kernel_size, pool_size, activation, padding, connecting_layer, pool_layer=True):
#     conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(connecting_layer)
#     conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(conv)
#     if pool_layer:
#       pool = keras.layers.MaxPooling2D(pool_size)(conv)
#       return conv, pool
#     else:
#       return conv

#   def __UpConvBlock(self, filters, up_filters, kernel_size, up_kernel, up_stride, activation, padding, connecting_layer, shared_layer):
#     conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(connecting_layer)
#     conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(conv)
#     up = keras.layers.Conv2DTranspose(filters=up_filters, kernel_size=up_kernel, strides=up_stride, padding=padding)(conv)
#     up = keras.layers.concatenate([up, shared_layer], axis=3)

#     return conv, up

In [ ]:
# model = inpaintingModel().prepare_model(input_size=(208, 176, 3))
# model.compile(optimizer='adam', loss='mean_absolute_error')

# history = model.fit(train_gen, 
#           validation_data=test_gen, 
#           epochs=20, 
#           steps_per_epoch=len(train_gen), 
#           validation_steps=len(test_gen),
#           use_multiprocessing=True)

In [ ]:
# plt.imshow(model.predict(np.array([test_gen[1][0][61]])).reshape(208, 176, 3))